In [ ]:
import sklearn.model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
dataset = dataset.drop(columns=['Predicted'])
cat_columns = ['cat_feature1','cat_feature2','cat_feature3','cat_feature4','cat_feature5']
target_name = 'target'
dataset_encoded = pd.get_dummies(dataset, columns= cat_columns)
X_train = dataset_encoded.drop(columns = target_name)
y_train = dataset_encoded[target_name]
class xgb_algorithm:
    def __init__(self, n_estimator, max_Tree_depth, learning_rate) -> None:
        self.n_estimators = n_estimator
        self.max_depth = max_Tree_depth
        self.learning_rate = learning_rate
        self.trees = []
    def fit(self, X, y):
        self.init_pred = np.mean(y)
        y_pred = np.full_like(y, self.init_pred)
        #X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(dataset.drop(columns =target_name), dataset[target_name], test_size=0.25, random_state=42)
        for i in range(self.n_estimators):
            print('Building tree number: ', i)
            y_train = dataset_encoded[target_name] - y_pred
            # Initialize the Decision Tree classifier
            clf = DecisionTreeClassifier(random_state = 42, max_depth= self.max_depth)
            # Train the classifier on the training data
            clf.fit(X_train, y_train)
            # Make predictions on the testing data
            y_pred = clf.predict_proba(X_train)
